Nella cartella *files* trovi 8 file:
- 2 file di testo
- 2 file audio
- 4 immagini,

con varie estensioni.

**Step 1**

Inizia creando, in un notebook, uno script Python che iteri in ordine alfabetico sui file della cartella files e, a seconda del tipo (audio, documento, immagine), li sposti nella relativa sottocartella (qui sotto trovi un esempio). Se la sottocartella non esiste, il tuo script dovrà crearla automaticamente.

Durante il ciclo, lo script deve stampare le informazioni dei file: nome, tipo e dimensione in byte. Questo è l'output desiderato:

In [1]:
import os
import csv 
import shutil

from PIL import Image
import numpy as np
from tabulate import tabulate

In [2]:
#Set the variables

In [3]:
path = os.getcwd()
if 'files' in os.listdir(path):
    path = os.path.join(path, 'files')

In [4]:
type_dict = [
    {'fold':'audio', 'f_ext': '.mp3'},
    {'fold':'doc', 'f_ext': '.txt'},
    {'fold':'doc', 'f_ext': '.odt'},
    {'fold':'images', 'f_ext': '.png'},
    {'fold':'images', 'f_ext': '.jpg'},
    {'fold':'images', 'f_ext': '.jpeg'}
]

In [5]:
dir_list = [file for file in os.listdir(path) if not file.startswith('.')]
dir_list.sort()

For each file: 

1. check if the folder exists, otherwise create it, 
2. print the information,
3. check if there is a file with the same name and if so, change the destination name 
4. move the file, 
5. update the recap.csv file

In [6]:
for file in [file for file in dir_list if os.path.isfile(os.path.join(path, file))]:
    file_path = os.path.join(path, file)
    
    file_extension = os.path.splitext(file_path)[1]
    
    try:
        folder = next(item for item in type_dict if item['f_ext'] == file_extension)['fold']
    except:
        print("Sorry! I can't handle the file with extension ", file_extension)
    else:
                
        #check if the folder exists, otherwise create it
        if not os.path.isdir(os.path.join(path, folder)):
            os.makedirs(os.path.join(path, folder))
        
        #print the information
        print(file.rsplit(".", 1)[0] + 
              " type:" + folder + 
              " size: " + str(os.path.getsize(file_path)) + "B")   
        
        file_detail = file.rsplit(".", 1)[0] + "," + folder + "," + str(os.path.getsize(file_path))
    
        #move file
        source = os.path.join(path, file)
        destination_folder = os.path.join(path, folder)
        
        #check if there is a file with the same name and if so, change the destination name
        count = 1
        while os.path.exists(os.path.join(destination_folder, file)):
            file = f"{file.rsplit('.', 1)[0]}({count}){file_extension}"
            count += 1
        
        destination = os.path.join(destination_folder, file)
        
        shutil.move(source, destination)
        
        #write detail into recap.csv
        insert_header = False
        if not os.path.exists(os.path.join(os.getcwd(), 'recap.csv')):
            insert_header = True
        
        with open('recap.csv', 'a', newline='') as csvfile: 
            csvwriter = csv.writer(csvfile) 
            if insert_header:
                csv_columns = ['name', 'type', 'size (B)']
                csvwriter.writerow(csv_columns)
            csvwriter.writerow(file_detail.split(','))

bw type:images size: 94926B
ciao type:doc size: 12B
ciao2 type:doc size: 12B
daffodil type:images size: 24657B
eclipse type:images size: 64243B
pippo type:doc size: 8299B
song1 type:audio size: 1087849B
song2 type:audio size: 764176B
trump type:images size: 10195B


Oltre a stamparne le informazioni via via che li sposti, tieni traccia dei file creando un documento *recap.csv* con le stesse informazioni. Trovi un esempio in questa cartella.

La struttura finale della cartella files dovrà essere:

        - files            
            - audio
                - song1.mp3
                - song2.mp3
            - docs
                - ciao.txt
                - pippo.odt
            - images
                - bw.png
                - daffodil.jpg
                - eclipse.png
                - trump.jpeg    
            - recap.csv

Commenta il codice con i passaggi che fai. Questo vale anche per i prossimi Step.

**Attenzione**: lo script, ogni volta che viene lanciato per spostare nuovi file, deve *aggiornare* (e non sovrascrivere) le sottocartelle e il file di recap. Per controllare che tutto funzioni correttamente, puoi aggiungere altri file alla cartella files e fare un test; oppure, puoi dividere gli 8 file originali in due gruppi e lasciarne uno per il test.

**Consiglio**: puoi usare le librerie *os*, *shutil* e *csv*. 
                
---

**Step 2**

Inserisci lo script che hai creato in un piccolo eseguibile (chiamalo *addfile.py* e posizionalo in questa cartella, a fianco del notebook) dotato di *interfaccia a linea di comando* (CLI).

Lo scopo dell'eseguibile è spostare un *singolo* file (che si trova nella cartella files) nella sottocartella di competenza, aggiornando il recap.

L'interfaccia dell'eseguibile ha come unico argomento (obbligatorio) il nome del file da spostare (comprensivo di formato, es: 'trump.jpeg'). Nel caso in cui il file passato come argomento non esista, l'interfaccia deve comunicarlo all'utente.

**Consiglio**: oltre alle precedenti, puoi usare le librerie *sys* e *argparse*.

---

**Step 3**

Una immagine in scala di grigio ha un solo livello di colore, una RGB ne ha 3, una RGBA 4 (l'ultimo è detto canale *alpha*).

Il modulo *Image* della libreria *PIL* permette di caricare un'immagine, che può essere trasformata in un array NumPy attraverso la funzione *np.array*. A partire da tale array, è possibile capire se l'immagine caricata è in scala di grigio, RGB o RGBA.

Aggiungi al notebook dello Step 1 uno script che iteri sulla sottocartella *images* e costruisca una tabella riassuntiva come questa (prodotta con la libreria *tabulate*):

In [7]:
table = [['name', 'height', 'width', 'grayscale', 'R','G','B', 'ALPHA']]

In [8]:
#Get detail about images into 'images' folder

In [9]:
image_path = os.path.join(path, 'images')
image_list = os.listdir(image_path)
image_list = [img for img in image_list if not (img.startswith('.'))]
image_list.sort()

In [10]:
for img in image_list:
    img_path = os.path.join(image_path, img)
    
    with Image.open(img_path) as im:
        a = np.asarray(im)
        grayscale, alpha = 0.0, 0.0
        r, g, b = 0.0, 0.0, 0.0
        
        if len(a.shape) == 2:
            grayscale = np.mean(a)
        else:
            if a.shape[2] == 4:
                alpha = a[:, :, 3].mean()
                
            r = np.mean(a, axis=(0, 1))[0]
            g = np.mean(a, axis=(0, 1))[1]
            b = np.mean(a, axis=(0, 1))[2] 
        
        row = [[(img.split('.')[0]), a.shape[0], a.shape[1], grayscale, r, g, b, alpha]]
        table += row

In [11]:
print(tabulate(table, headers="firstrow", tablefmt="fancy_grid", floatfmt=".2f"))

╒══════════╤══════════╤═════════╤═════════════╤════════╤════════╤═══════╤═════════╕
│ name     │   height │   width │   grayscale │      R │      G │     B │   ALPHA │
╞══════════╪══════════╪═════════╪═════════════╪════════╪════════╪═══════╪═════════╡
│ bw       │      512 │     512 │       21.48 │   0.00 │   0.00 │  0.00 │    0.00 │
├──────────┼──────────┼─────────┼─────────────┼────────┼────────┼───────┼─────────┤
│ daffodil │      500 │     335 │        0.00 │ 109.25 │  85.56 │  4.97 │    0.00 │
├──────────┼──────────┼─────────┼─────────────┼────────┼────────┼───────┼─────────┤
│ eclipse  │      256 │     256 │        0.00 │ 109.05 │ 109.52 │ 39.85 │  133.59 │
├──────────┼──────────┼─────────┼─────────────┼────────┼────────┼───────┼─────────┤
│ trump    │      183 │     275 │        0.00 │  97.01 │  98.99 │ 90.92 │    0.00 │
╘══════════╧══════════╧═════════╧═════════════╧════════╧════════╧═══════╧═════════╛


Oltre al nome del file, la tabella riporta:

- altezza dell'immagine, in pixel
- larghezza dell'immagine, in pixel
- se l'immagine è in scala di grigio, la colonna *grayscale* indica la media dei valori dell'unico livello di colore
- se l'immagine è a colori, le altre colonne indicano la media dei valori di ogni livello di colore.

---

**Dovrai consegnare**:
- un notebook con gli Step 1 e 3; per semplicità puoi chiamarlo come questo
- addfile.py con quanto richiesto dallo Step 2.